# Projeto

banco de dados: 
https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset

Pacotes necessários:
- ``os``
- ``translate``
- ``json``
- ``csv``
- ``langdetect``

## Função para validação de respostas do usuário

In [679]:
def validacao_respostas(pergunta, respostas_validas):
  """
  Validação de respostas. A função valida respostas de um caracter
  É passado como parâmetro:
    - A pergunta que se quer fazer (str)
    - Uma lista com as respostas válidas

  Caso o usuário digite algo diferente das respostas válidas,
  a função repetirá a pergunta até que o usuário responda um valor válido.
  E só retornará opções válidas de valores
  """
  print(pergunta)
  resp = input().upper()[0]# apenas 1ªletra
  pergunta = 'Resposta inválida! ' + pergunta
  while not resp in respostas_validas:
    print(pergunta)
    resp = input().upper()[0]# apenas 1ªletra
    #resp = input(pergunta).upper()[0]# apenas 1ªletra
  return resp

## Função que lista os arquivos disponíveis

In [680]:
import os
def arquivos_disponiveis(extensao: str):
    """Função recebe uma extensão de arquivo e lista todos os arquivos com a extensão que estão no diretório

    Args:
        extensao (str): extensão do arquivo

    Returns:
        opcoes (dict): as opções de arquivos disponíveis
    """
    #pergunta = '|Informe o numero do arquivo que deseja importar|'
    opcoes = dict()
    arquivos = os.listdir() # lista todos os arquivos presentes na pasta
    arquivos_extensao = [arquivo for arquivo in arquivos if arquivo.endswith(extensao)] # filtra pela extensao
    print(f'        | Lista de arquivos {extensao.upper()} disponíveis |')
    for i in range(len(arquivos_extensao)):
        print(f'\t{i+1}\t{arquivos_extensao[i]}')
        opcoes.update({str(i+1):arquivos_extensao[i]})
    return opcoes
        

# Função que retorna uma lista de valores sem repetição

In [681]:
def lista_de_valores_sem_repeticoes(lista_de_dicionarios, chave):
  """
  Recebe uma lista de dicionários.
  A função retorna lista dos valores encontrados para a chave, sem repetições de valores.

  Args:
    lista_de_dicionarios: Uma lista de dicionários.
    chave: A chave a ser pesquisada.

  Returns:
    Uma lista.
  """
  lista_de_valores = set()
  for dicionario in lista_de_dicionarios:
    if chave in dicionario:
      lista_de_valores.add(dicionario[chave])
  return list(lista_de_valores)

# Verifica se a variável é um número

In [682]:
def validacao_numero(pergunta:str, var_tipo: type):
    """Valida e converte variáveis numéricas

    Args:
        pergunta (str): Pergunta realizada pelo usuário para coletar o valor
        var_tipo (type): tipo de variável numérica do valor que será informado pelo usuário

    Returns:
        valor validado e convertido de acordo com a variável var_tipo
    """
    valido = False
    print(pergunta)
    if var_tipo == int:
        while valido != True:
            numero = input(pergunta)
            try:
                int(numero)
                valido = True
            except ValueError:
                print("Informe um valor válido!")
                numero = input(pergunta)
                valido = False
            else:
                return int(numero)
    else:
        while valido != True:
            numero = input(pergunta)
            try:
                float(numero)
                valido = True
            except ValueError:
                print("Informe um valor válido!")
                numero = input(pergunta)
                valido = False
            else:
                return float(numero)



# Funções de Importação

## Função para a leitura de dados de um csv ou json e transformação em uma lista de dicionários

In [683]:
import json
import csv
def importar_dados(extensao:str):
    """ realiza a importação dos dados de arquivos .json ou .csv

    Args:
        extensao (str): extensão do arquivo

    Returns:
        dados(list): lista de dicionários com as variáveis selecionadas para o estudo de caso
    """
    dados = list()
    
    opcoes = arquivos_disponiveis(extensao)
    respostas_validas = list(opcoes.keys())
    pergunta = """
    ╔═════════════════════════════════════════════════════╗
    ║   Informe o número do arquivo que deseja importar   ║
    ╚═════════════════════════════════════════════════════╝
    """
    str_arquivo = opcoes[validacao_respostas(pergunta, respostas_validas)]
    
    if extensao == '.json':
        with open(str_arquivo, "r", encoding = "utf-8") as f:
            dados = json.load(f) # os arquivos importados via .json já estarão filtrados pelas categorias filtradas pela importação via .csv

    else:
        with open(str_arquivo, "r") as arquivo:
            arquivo_csv = csv.reader(arquivo, delimiter=",")
            next(arquivo_csv)  # pula a primeira linha (cabeçalho)
            for linha in enumerate(arquivo_csv):
                Customer_ID, Age, Gender, Item_Purchased,Category, Purchase_Amount, Location, Size, Color, Season, Review_Rating, Subscription_Status, Payment_Method, Shipping_Type, Discount_Applied, Promo_Code_Used, Previous_Purchases, Preferred_Payment_Method, Frequency_of_Purchases = linha[1]
                #print('ID', Customer_ID,'Idade', Age,'Genero', Gender,'Item',Item_Purchased,
                #                          'Categoria', Category,'Metodo_Pagamento', Payment_Method)
                # 
                dados.append({      'ID': int(Customer_ID), 
                                    'Idade': int(Age), #numérica
                                    'Genero': Gender, #categórica
                                    'Item':Item_Purchased, #categórica
                                    'Categoria': Category, #categórica
                                    'Preco': float(Purchase_Amount), #numérica
                                    'Metodo_Pagamento': Payment_Method, #categórica
                                    'Qtd_Compras': int(Previous_Purchases)#, #numérica
                                    #'Frequencia_Compra': Frequency_of_Purchases #categórica
                                    })
    return dados


## Função para traduzir palavras
É necessário instalar o pacote ``translate``

In [684]:
# (terminal) pip install translate
from translate import Translator
def traduz_palavras(lista_de_palavras):
  """
  Recebe uma lista de palavras em inglês e retorna um dicionário onde 
  a chave é a palavra da lista e o valor é a tradução em português.

  Args:
    lista_de_palavras: Uma lista de palavras em inglês.

  Returns:
    Um dicionário.
  """
  dicionario_de_traducao = {}
  traducao = Translator(to_lang='pt-br', from_lang='english')
  for palavra in lista_de_palavras:
    dicionario_de_traducao[palavra] = traducao.translate(palavra)
  return dicionario_de_traducao


## Traduzindo os valores retirados do CSV

In [685]:
def traduzindo_lista_dict(lista: list):
    """Traduz os valores de uma lista de dicionários

    Args:
        lista (list): lista de dicionários

    Returns:
        lista (list): lista de dicionários traduzida
    """
    for chave in list(lista[0].keys()):
        if type(lista[0][chave]) == str:
            categorias = lista_de_valores_sem_repeticoes(lista, chave)
            dict_traducao = traduz_palavras(categorias)
            for i in range(len(lista)):

                lista[i][chave] = dict_traducao[lista[i][chave]]
    return lista

## Detectar idioma

In [686]:
from langdetect import detect 

def detectar_idioma(dados:list):
    """ A função detecta o idioma dos dados importados e
    questiona ao usuário que ser traduzí-los.
    Se a resposta for positiva, a função traduzindo_lista_dict() será chamada.

    Args:
        dados (list): dados dispostos em listas de dicionários

    Returns:
        dados (list): dados dispostos em listas de dicionários traduzidos para português
    """
    if detect(dados[0]['Metodo_Pagamento']) == 'en': # um exemplo de dados categóricos para verificar o idioma
        pergunta = """
        ___________________________________________
        Os dados estão em inglês, deseja traduzir?
        [S] Sim  [N] Não\t
        ____________________________________________"""
        resp = validacao_respostas(pergunta, ['S','N'])
        if resp == 'S':
            print("Essa operação pode demorar, aguarde.")
            return traduzindo_lista_dict(dados)
        return dados



## Menu de importação dos dados

In [687]:
def menu_importar_dados():
    """
        A função coleta do usuário as informações necessárias para realizar a importação de dados.
        Informações coletadas:
        - a extensão do dado que deseja.
            - csv
            - json
            - não importar nenhum arquivo
        - o nome do arquivo

    Return:
        dados (list): dados dispostos em listas de dicionários  
    
    """
    pergunta = """
    ╔═════════════════════════════════════╗
    ║    Bem Vindo à Gestão de Dados      ║
    ║ Que tipo de arquivo deseja importar:║
    ╠═════════════════════════════════════╣
    ║  1 \t CSV                      ║  
    ║  2 \t JSON                     ║
    ║  3 \t Não importar nada        ║ 
    ╚═════════════════════════════════════╝    
    
    """
    resp =  validacao_respostas(pergunta, ['1','2','3'])
    if   resp == '1':
        dados = importar_dados('.csv')
        detectar_idioma(dados)
    elif resp == '2':
        dados = importar_dados('.json')
        detectar_idioma(dados)
    else:
        dados = list()
    return dados

# Função para exportar uma lista de dicionários para um arquivo .json

In [688]:
import json
import os.path
def exportar_json(lista_vendas: list):
    """
    A função exporta a lista_vendas para um arquivo.
    O arquivo será nomeado pelo usuário
        Caso o arquivo já exista na pasta, o usuário será questionado se quer sobrescrever.
    O arquivo será no formato .json
    Será importado na mesma pasta que se encontra o script
    Args:
        lista_vendas (list): lista de dicionários com as informações de venda
    """
    nome_arquivo = input("Informe o nome do arquivo a ser criado:\t").lower()
    

    # retirando os possíveis espaços do nome
    qtd_espaco = nome_arquivo.count(' ')
    if qtd_espaco > 0:
        nome_arquivo =  nome_arquivo.replace(" ", "_", qtd_espaco)
    nome_arquivo += '.json'
    
    if os.path.exists(nome_arquivo) == True:
        pergunta = 'Já existe um arquivo com esse nome.\nDeseja sobrescrever?[S]  Sim\t [N] Não\t'
        resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
        if resp_sobrescrever == 'N':
            print('O arquivo não foi modificado.')
            resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
            return
    try:
        json_vendas = json.dumps(lista_vendas, indent=2,ensure_ascii=False)
        arquivo = open(nome_arquivo, 'w', encoding = "utf-8") 
        arquivo.write(json_vendas) 
        arquivo.close()
        print("Arquivo exportado com sucesso!")
    except:
        print("Erro ao exportar arquivo")
    return

# Listagem de dados

## Lista simples de todos os dados da lista

In [689]:
def listar_dados(dados: list):
    """Imprime na tela uma lista simples com um registro por linha

    Args:
        dados (list): lista de dicionários
    """
    print("\t".join(list(dados[0].keys())))
    for j in range(len(dados)):
        lista_linha = [str(list(dados[j].values())[i]) for i in range(len(list(dados[j].values())))]
        print("\t".join(lista_linha))


## Função que exibe os registros

In [690]:
def exibir_dados(dados: list, ids: list ):
    """Exibe os dados de maneira detalhada, cada chave em uma linha

    Args:
        dados (list): lista de dicionários
        ids (list): identificadores dos registros
    """
    lista_indices = [i for i in range(len(dados)) if dados[i]['ID'] in ids]
    print("Listagem:")
    lista_chaves = list(dados[0].keys())
    for i in lista_indices:
        print(' ===========================================')
        lista_valores = list(dados[i].values())
        for j in range(len(lista_chaves)):
            print(f'{lista_chaves[j].upper()}: {lista_valores[j]}')

## Menu de listagem de registro

In [691]:
def menu_lista(dados: list):
    """Solicita do usuários informações para a listagem dos dados

    Args:
        dados (list): lista de dicionários
    """
    pergunta ='''
    =================================
          Opções de Listagem:
    =================================      
    [1] Lista de todos os registros
    [2] Lista de registros específicos - via ID
    [3] Lista de registros com ids sequenciais
    =================================
    '''
    resp = validacao_respostas(pergunta, ['1','2','3'])
    if resp == '1':
        listar_dados(dados)
    elif resp == '2':
        pergunta = "Informe o ID que seja visualizar registro e 0 para sair"
        numero = 1# para inicializar o while
        lista_ids = list()
        while True:
            numero = validacao_numero(pergunta, int)
            if numero == 0:
                break
            lista_ids.append(numero)
        if len(lista_ids) == 0:
            print('Não há registros para exibir!')
        else:
            exibir_dados(dados,lista_ids)
    else:
        pergunta = "Informe o ID inicial da sequência:"
        numero_inicial = validacao_numero(pergunta, int)
        while numero_inicial <=0:
            print('Informe valores positivos de ID')
            numero_inicial = validacao_numero(pergunta, int)
            
        pergunta = "Informe o ID final da sequência:"
        numero_final = validacao_numero(pergunta, int)
        while numero_final <= numero_inicial:
            print('Informe um valor maior que o ID inicial')
            numero_final = validacao_numero(pergunta, int)
        exibir_dados(dados,list(range(numero_inicial, numero_final)))
        

# Funções de Editar registros

## edição do registro

In [692]:
def editar_registros(dados: list, chave: str, valor, id:int):
    """A função recebe as informações para editar o registro e retorna os dados atualizados

    Args:
        dados (list): lista de dados
        chave (str): chave das categorias do registro
        valor (_type_): valor do registro
        id (int): identificador do registro

    Returns:
        dados (list): lista de dicionários atualizado
    """
    indice =  [i for i in range(len(dados)) if dados[i]['ID'] == id][0]
    dados[indice][chave] = valor
    return dados

## seleção da chaves que se quer editar

In [693]:
def selecao_chave_edicao(dados: list, id : int):
    """Solicita do usuário a informação dos campos que deseja editar

    Args:
        dados (list): lista de dicionários
        id (int): identificador do registro

    """
    
    
    print('Qual ou quais campos deseja editar, separando-os por vírgula')
    indice =  [i for i in range(len(dados)) if dados[i]['ID'] == id][0]
    chaves = list(dados[indice].keys())
    lista_opcoes = [1]
    i = 1
    for chave in chaves[1:]:
        print(f'{i}\t{chave}')
        i +=1
        lista_opcoes.append(i)
    print(f'{i}\tNovo campo')
    
    lista_selecionados     = (input().replace(' ','')).split(',')
    lista_opcoes_str       = list(map(str, lista_opcoes))
    
    errados = [i for i in lista_selecionados if not i in lista_opcoes_str]
    if len(errados) >= 1:
        if len(errados) == 1:
            print(f'O campo "{" ".join(errados)}" foi desconsiderado por ser inválido.')
        else:
            print(f'Os campos\n{" - ".join(errados)}\nforam desconsiderados por serem inválidos.')

        pergunta = 'Deseja refazer a seleção dos campos?\n[S] Sim\n[N] Não'
        resp = validacao_respostas(pergunta, ['S','N'])
    
        if resp == 'S':
            selecao_chave_edicao(dados, id)
    
    lista_selecionados_corrigidos = [i for i in lista_selecionados if not i in errados  ]
    lista_selecionados_corrigido_int = list(map(int, lista_selecionados_corrigidos ))
    if max(lista_selecionados_corrigido_int) == len(chaves):
        novo_campo = input("Foi selecionado a opção 'Novo Campo'\n Nomeei novo campo:")
        chaves.append(novo_campo)
    
    print('Campos selecionados para a edição:')
    for i in lista_selecionados_corrigido_int:
        chave = chaves[i]
        pergunta = f'Informe o novo valor para o campo {chave.title()}'
        if dados[indice][chave] == None:
            pergunta = f'''Que tipo de informação será armazenada no campo {chave}?
            [1] Categórico
            [2] Números Inteiros
            [3] Números Decimais'''
            resp = validacao_respostas(pergunta, ['1','2','3'])
            if   resp == '2':
                valor = validacao_numero(pergunta, int)
            elif resp == '3':
                valor = validacao_numero(pergunta, float)
            else:
                valor = input(pergunta).title()
        elif type(dados[indice][chave]) == int:
            valor = validacao_numero(pergunta, int)
        elif type(dados[indice][chave]) == float:
            valor = validacao_numero(pergunta, float)
        else:
            if chave == 'Genero':
                print('''Gênero:
          [1] Homem
          [2] Mulher
          ''')
                genero = validacao_respostas("Gênero: ", ['1', '2'])
                if genero == '1':
                    valor = 'Homem'
                else:
                    valor = 'Mulher'
                    
            elif chave == 'Categoria':
                categorias = ['Vestuário','Acessórios','Calçado','Agasalhos']
                pergunta = 'Categoria: '
                print(pergunta)
                for i in range(len(categorias)):
                    print(f'{i+1}\t{categorias[i]}')
                valores_validos = list(map(str, list(range(1,i+2))))
    
                num_categoria = validacao_respostas(pergunta, valores_validos)
                num_categoria = int(num_categoria)-1
                valor = categorias[num_categoria]
                
            elif chave == 'Metodo_Pagamento':
                metodos_pagamento = ['Cartão de Crédito', 'Venmo','dinheiro',
                        'PayPal','Cartão de Débito','Transferência Bancária']
                pergunta = 'Método de Pagamento: '
                print(pergunta)
                for i in range(len(metodos_pagamento)):
                    print(f'{i+1}\t{metodos_pagamento[i]}')
                valores_validos = list(map(str, list(range(1,i+2))))
                
                num_pg = validacao_respostas('Método de Pagamento:', valores_validos)
                num_pg = int(num_pg)-1
                valor = metodos_pagamento[num_pg]

            else:
                valor = input(pergunta).title()
        return editar_registros(dados, chave, valor, id)
    print('O registro foi atualizado!')
    exibir_dados(dados, [id])
    return dados


## seleção do registro que se quer alterar

In [694]:
def selecao_id_edicao(dados: list):
    """Solicita do usuário o identificador do registro que deseja alterar

    Args:
        dados (list): lista de dicionários

    """
    print('Para fazer a edição de um registro, é necessário informar o seu ID')
    pergunta = '''Deseja listar os registros?
    [S]\t Sim
    [N]\t Não
    '''
    resp = validacao_respostas(pergunta, ['S','N'])
    if resp == 'S':
        listar_dados(dados)
    pergunta = 'Informe o ID do registro que deseja editar'
    
    id_valido = list(map(str,list(set(item['ID'] for item in dados))))
    resp = validacao_respostas(pergunta, id_valido)
    return selecao_chave_edicao(dados, resp)

# Função Remover

In [695]:
import json
def remover_arquivo(dados):
    """"
    Remove dicionários de um arquivo json através do ID inserido.

    Parâmetros:
    dados(list): Uma lista de dicionários.
    """

    id_input = 'Digite o ID que deseja remover: ' 
    id_remover = validacao_numero(id_input, int)   

    dados2 = [item for item in dados if item['ID'] != id_remover]

    if len(dados2) < len(dados):
        exibir_dados(dados, [id_remover])
        deseja_remover = 'Tem certeza que deseja remover essas informações? [S] Sim  [N] Não\t '
        resposta = validacao_respostas(deseja_remover, ['S', 'N'])

        if resposta == 'S':
            print('Dados removidos com sucesso!')
                
            pergunta = '''O que deseja fazer com os dados filtros:
            [1] Exibir na tela
            [2] Exportar
            [3] Voltar para o menu
            '''
            resp = validacao_respostas(pergunta, ['1','2','3'])
            if resp == '1':
                listar_dados(dados2)
            elif resp == '2':
                exportar_json(dados2)
            return dados2
        
        else:
            print('Dados recuperados')
            return dados
    
    else:
        print(f'O ID {id_remover} não foi encontrado!')

# Funções para filtro

## Função de filtro

In [696]:
def filtro(lista: list, categoria:str, valor_categoria, tipo:str):
    """Função que realiza os filtros de acordo com os parâmetros de entrada passados

    Args:
        lista (list): dados dispostos em listas de dicionários
        categoria (str): chave da categoria que se deseja filtrar 
        valor_categoria: valor utilizado com o operador comparativo
        tipo (str): nome do operador comparativo

    Returns:
        filtro (list): dados filtrados dispostos em listas de dicionários
    """

    if   tipo == 'Maior que':
        filtro = list(filter(lambda x: x[categoria] >  valor_categoria, lista))
    elif tipo == 'Maior ou igual':
        filtro = list(filter(lambda x: x[categoria] >= valor_categoria, lista))
    elif tipo == 'Menor que':
        filtro = list(filter(lambda x: x[categoria] <  valor_categoria, lista))
    elif tipo == 'Menor ou igual':
        filtro = list(filter(lambda x: x[categoria] <= valor_categoria, lista))
    elif tipo == 'Igual a':
        filtro = list(filter(lambda x: x[categoria] == valor_categoria, lista))
    else:       # Diferente de
        filtro = list(filter(lambda x: x[categoria] != valor_categoria, lista))
    return filtro


### Filtro Categórico

In [697]:
def filtro_categorico(dados:list, chave:str):
    """A função coleta do usuário qual operação se deseja realizar no filtro de variáveis categóricas

    Args:
        dados (list): dados dispostos em listas de dicionários
        chave (str): chave da categoria que se deseja filtrar
        As opções de filtros são
                - Diferente de
                - Igual a
    Returns:
        dados   (list): dados dispostos em listas de dicionários
        chave    (str): chave da categoria que se deseja filtrar  
        valor    (str): valor para realizar a operação de comparação 
        operacao (str): Nome da operação
    """

    pergunta = f'Informe o número da opção você quer filtrar dentro de {chave}: '
    opcoes = lista_de_valores_sem_repeticoes(dados, chave)

    respostas_validas = list(range(1,len(opcoes)+1))
    respostas_validas = list(map(str, respostas_validas))
    print(f'\nLista de opções dentro da categoria {chave}')
    for i in range(len(opcoes)):
        print(1+i,'\t',opcoes[i])

    resp = validacao_respostas(pergunta, respostas_validas)
    valor = opcoes[int(resp)-1]
    pergunta = f'''Como deseja filtrar os dados dentro de {chave}
        [1] Para valores diferentes de {valor}
        [2] Para valores iguais a de {valor}.'''
    resp = validacao_respostas(pergunta, ['1','2'])

    if resp == '1':
        operacao = 'Diferente de'
    else:
        operacao = 'Igual a'
    return dados, chave, valor, operacao

### Filtro Numérico

In [698]:
def filtro_numerico(dados:list, chave:str, var_tipo: type):
    """A função coleta do usuário qual operação se deseja realizar no filtro de variáveis numéricas

    Args:
        dados (list): dados dispostos em listas de dicionários
        chave (str): chave da categoria que se deseja filtrar
        var_tipo(type): tipo da variável (int ou float) que será utilizada para a comparação
        As opções de filtros são
                - Diferente de
                - Igual a
                - Maior que
                - Maior ou Igual
                - Menor que
                - Menor ou Igual 
    Returns:
        dados   (list): dados dispostos em listas de dicionários
        chave    (str): chave da categoria que se deseja filtrar  
        valor  (float): valor para realizar a operação de comparação 
        operacao (str): Nome da operação
    """
    operadores_comparacao = ['Maior que','Maior ou igual',
                            'Menor que','Menor ou igual',
                            'Igual a','Diferente de']
    respostas_validas = list(range(1,len(operadores_comparacao)+1))
    respostas_validas = list(map(str, respostas_validas))

    for i in range(len(respostas_validas)):
        print(1+i,'\t',operadores_comparacao[i])
    pergunta = f'Informe o número da opção do operador para usar na categoria {chave}: '
    resp = validacao_respostas(pergunta, respostas_validas)
    operacao = operadores_comparacao[int(resp)-1]
        
    pergunta = 'Informe um valor para realizar a comparação: '
    valor = validacao_numero(pergunta, var_tipo)
    print(chave, valor, operacao)
    return dados, chave, valor, operacao

## Menu filtros

In [699]:
def opcoes_filtros(dados:list):
    """A função coleta do usuário as informações necessárias para realizar a filtragem de dados.
        Informações coletadas:
        A categoria (chave) que se quer usar para realizar o filtro.
        A operação que se deseja realizar será coletada a partir de duas funções
        - filtro_categorico para variáveis categóricas
        - filtro_numerico para variáveis numéricas (float e int)
        E o valor necessário para tal.
    Args:
        dados (list): dados dispostos em listas de dicionários
    """
    tamanho = len(list(dados[0].keys())[1:])
    lista_categorias = list(dados[0].keys())[1:]
    respostas_validas = list(range(1,tamanho+1))
    respostas_validas = list(map(str, respostas_validas))

    print('\nCategorias Disponíveis para consulta:')
    for i in range(tamanho):
        print(1+i,'\t',lista_categorias[i])
    pergunta = 'Informe o número da categoria que deseja filtrar:'
    resp = validacao_respostas(pergunta, respostas_validas)
    chave = str(lista_categorias[int(resp)-1])

    if type(dados[0][chave]) == str:
        dados, chave, valor, operacao = filtro_categorico(dados, chave)
          
    elif type(dados[0][chave]) == int: 
        dados, chave, valor, operacao = filtro_numerico(dados, chave, int)
    else: # float
        dados, chave, valor, operacao = filtro_numerico(dados, chave, int)
        
    print(f'''Detalhes do filtro:
              Categoria: {chave}
              Valor:     {valor}
              Operação:  {operacao}
              ''')
    dados_filtrados = filtro(dados, chave, valor, operacao)
    
    pergunta = '''O que deseja fazer com os dados filtros:
    [1] Exibir na tela
    [2] Exportar
    '''
    resp = validacao_respostas(pergunta, ['1','2'])
    if resp == '1':
        listar_dados(dados_filtrados)
    else:
        exportar_json(dados_filtrados)

# Função adicionar registro

In [700]:
def adicionar_dados_manualmente(dados:list):
    """Adiciona todos os campos iniciais de um registro e o adiciona na lista de registros

    Args:
        dados (list): lista de dicionários

    Returns:
        dados (list): lista de dicionários atualizados
    """
    if len(dados) == 0:
        novo_id = 1

    else:
        novo_id = dados[-1]['ID'] + 1

    categorias = ['Vestuário','Acessórios','Calçado','Agasalhos']
    metodos_pagamento = ['Cartão de Crédito', 'Venmo','dinheiro',
                        'PayPal','Cartão de Débito','Transferência Bancária']

    idade = validacao_numero("Idade: ", int)
    
    print('''Gênero:
          [1] Homem
          [2] Mulher
          ''')
    genero = validacao_respostas("Gênero: ", ['1', '2'])
    if genero == '1':
        genero = 'Homem'
    else:
        genero = 'Mulher'
        
    item = input("Item: ")
    
    pergunta = 'Categoria: '
    print(pergunta)
    for i in range(len(categorias)):
        print(f'{i+1}\t{categorias[i]}')
    valores_validos = list(map(str, list(range(1,i+2))))
    
    num_categoria = validacao_respostas('Categoria:', valores_validos)
    num_categoria = int(num_categoria)-1
    categoria = categorias[num_categoria]
    
    preco = validacao_numero("Preço: ", float)
    
    
    pergunta = 'Método de Pagamento: '
    print(pergunta)
    for i in range(len(metodos_pagamento)):
        print(f'{i+1}\t{metodos_pagamento[i]}')
    valores_validos = list(map(str, list(range(1,i+2))))
    
    num_pg = validacao_respostas('Método de Pagamento:', valores_validos)
    num_pg = int(num_pg)-1
    metodo_pagamento = metodos_pagamento[num_pg]

    
    
    qtd_compras = validacao_numero("Quantidade de Compras: ", int)

    novo_dado = {
        'ID': novo_id,
        'Idade': idade,
        'Genero': genero,
        'Item': item.title(),
        'Categoria': categoria,
        'Preco': preco,
        'Metodo_Pagamento': metodo_pagamento,
        'Qtd_Compras': qtd_compras
    }

    
    dados.append(novo_dado)
    print("Novo registro adicionado com sucesso!")
    return dados

# Estatísticas

## Função para calcular ticket médio

In [701]:
def calcular_ticket_medio(dados):
    """
    O ticket médio é um indicador usado para avaliar o gasto médio dos clientes.
    Para calculá-lo, deve-se dividir o faturamento total das vendas pelo número de vendas do intervalo de tempo em questão.
    """
    faturamento = 0
    for item in dados:
        faturamento = sum([faturamento, item["Preco"]])
    
    transacoes = len(dados)

    if transacoes == 0:
        # Para evitar divisões por zero
        return 0
    else:
        ticket_medio = faturamento / transacoes
        return ticket_medio

## Regra de Negócio

In [702]:
def grupo_clientes(dados):
    """
    A clusterização de clientes permite agrupá-los por alguma semelhança.
    É um meio para entender o perfil de consumo dos clientes para que, de forma estratégica,
        sejam criados ofertas personalizadas.
    Nessa função os clientes serão separados em dois grupos:
    - A: perfil de consumo a partir do ticket médio
    - B: consome abaixo do ticket médio
    """
    ticket_medio = calcular_ticket_medio(dados)

    for dicionario in dados:
        if dicionario["Preco"] >= ticket_medio:
            dicionario["Grupo_Cliente"] = "A"
            
        else:
            dicionario["Grupo_Cliente"] = "B"
    return dados

In [703]:
def saida_ticket_medio(dados:list):
      """Exibe para o usuário o valor do ticket médio e o que ele representa.
      ao final, ocorre a chamada da função grupo_clientes() para inserir na 
      lista de dicionário com os dados das vendas a categoria de cada cliente

      Args:
          dados (list): lista de dicionários
      """

      ticket_medio = calcular_ticket_medio(dados)
      print(f'''Valor do Ticket Médio:\nR$ {ticket_medio:.2f}
          Pelo valor do Ticket Médio, podemos classificar os clientes em dois grupo:
          Grupo A: Perfil de consumo a partir do ticket médio
          Grupo B: Perfil de consumo abaixo do ticket médio
          ''')

      print('\nA classificação dos cliente pelos grupos foram adicionados no banco de dados')
      grupo_clientes(dados)
    

# Estatísticas

## Por genero

In [704]:
def estatistica_genero(dados):
    """Cálculo da quantidade total e percentual das vendas por genero.       
       A função informa na tela os valores extraídos dos dados

    Args:
        dados (list): Lista de dicionários
    """
    total_dicionarios = len(dados)
    
    # Filtrar os dicionários para obter apenas os homens
    homens = filtro(dados, "Genero", "Homem", "Igual a")

    # Filtrar os dicionários para obter apenas as mulheres
    mulheres = filtro(dados, "Genero", "Mulher", "Igual a")

    # Calcular o número de homens e mulheres
    n_homem = len(homens)
    n_mulher = len(mulheres)

    # Calcular percentuais
    percentual_homem = (n_homem / total_dicionarios) * 100
    percentual_mulher = (n_mulher / total_dicionarios) * 100

    # Exibir resultados
    print ('═══════════════════════════════════════════════ ')
    print("Número total de pessoas:", total_dicionarios)
    print ('═══════════════════════════════════════════════ ')
    print("Número de homens:", n_homem)
    print("Número de mulheres:", n_mulher)
    print ('═══════════════════════════════════════════════ ')
    print("Percentual de homens: {:.2f}%".format(percentual_homem))
    print("Percentual de mulheres: {:.2f}%".format(percentual_mulher))
    print ('═══════════════════════════════════════════════ ')

    return

## Por cliente

In [705]:
def estatistica_grupo_cliente(dados):
    """Cálculo da quantidade total e percentual das vendas por grupos de clientes.
       O grupo do cliente é determinado pelo valor do ticket médio
       Grupo A: valor gasto a partir do ticket médio
       Grupo B: Abaixo do valor médio
       
       A função informa na tela os valores extraídos dos dados

    Args:
        dados (list): Lista de dicionários
    """
    # inserindo a classificação dos clientes na variável que armazena as informações de vendas
    grupo_clientes(dados)
    total_dicionarios = len(dados)

    # Filtrar os dicionários para obter apenas os de Grupo A
    grupo_A = filtro(dados, "Grupo_Cliente", "A", "Igual a")

    # Filtrar os dicionários para obter apenas os de Grupo B
    grupo_B = filtro(dados, "Grupo_Cliente", "B", "Igual a")

    # Calcular o número de Grupo A e de Grupo B
    n_grupo_A = len(grupo_A)
    n_grupo_B = len(grupo_B)

    # Calcular percentuais
    percentual_a = (n_grupo_A / total_dicionarios) * 100
    percentual_b = (n_grupo_B /total_dicionarios) * 100

    # Exibir resultados
    print ('═══════════════════════════════════════════════ ')
    print("Total de clientes:", total_dicionarios)
    print ('═══════════════════════════════════════════════ ')
    print("Número de clientes no Grupo A:", n_grupo_A)
    print("Número de clientes no Grupo B:", n_grupo_B)
    print ('═══════════════════════════════════════════════ ')
    print("Percentual de clientes no Grupo A: {:.2f}%".format(percentual_a))
    print("Percentual de clientes no Grupo B: {:.2f}%".format(percentual_b))
    print ('═══════════════════════════════════════════════ ')

    return

## Cálculos de porcentagem de vendas pelas chaves com mais de duas categorias


In [706]:
def venda_por_chave(dados: list, chave:str):
  """_summary_

  Args:
      dados (list): _description_
      chave (str): _description_

  Returns:
      categoria_ordenada:(list): lista em ordem decrescente das categorias presentes na chave
  """
  # selecionando os valores únicos de chave
  categorias = list(set(item[chave] for item in dados)) 
  #inicializando o dicionário
  categorias_dicionario = dict.fromkeys(categorias, 0) 
  
  for categoria in categorias: #acrescentando os valores às chaves
    tot_categoria = len(filtro(dados, chave, categoria, 'Igual a'))
    categorias_dicionario[categoria] += tot_categoria

  # ordenando em forma decrescente
  categorias_ordenadas = sorted(categorias_dicionario.items(),
                                key=lambda item: item[1],
                                reverse=True)
  return categorias_ordenadas

## Tabela as estatísticas com mais de duas categorias

In [707]:
def saida_estatistica(dados:list, chave: str, limite :int= None):
    """Exibição dos resultados das estatísticas em formato de tabela

    Args:
        dados (list): lista de dicionários com os dados de vendas
        chave (str): categoria de dados que será feita a estatística
        limite (int, optional): Limite de linhas que será exibidas. Defaults to None.
    """

    qtd_por_venda = venda_por_chave(dados,chave)

    soma = sum(tupla[1] for tupla in qtd_por_venda)

    if limite == None or limite >= len(qtd_por_venda):
        print("\nItem\t\t\tQtd\t %")
        print ('═══════════════════════════════════════════════ ')
        for linha in qtd_por_venda:
            if len(linha[0]) <=7:
                print(f'{linha[0]}\t\t\t{linha[1]}\t{(linha[1]*100/soma):.2f}')
                print ('═══════════════════════════════════════════════ ')
            elif len(linha[0]) < 16:
                print(f'{linha[0]}\t\t{linha[1]}\t{(linha[1]*100/soma):.2f}')
                print ('═══════════════════════════════════════════════ ')
            else:
                print(f'{linha[0]}\t{linha[1]}\t{(linha[1]*100/soma):.2f}')
                print ('═══════════════════════════════════════════════ ')

    elif limite < len(qtd_por_venda):
        print("\nItem\t\t\tQtd\t %")
        printprint ('═══════════════════════════════════════════════ ')
        for linha in qtd_por_venda[:(limite+1)]:
            if len(linha[0]) < 7:
                print(f'{linha[0]}\t\t\t{linha[1]}\t{(linha[1]*100/soma):.2f}')
                print ('═══════════════════════════════════════════════ ')
            elif len(linha[0]) < 16:
                print(f'{linha[0]}\t\t{linha[1]}\t{(linha[1]*100/soma):.2f}')
                print ('═══════════════════════════════════════════════ ')
            else:
                print(f'{linha[0]}\t{linha[1]}\t{(linha[1]*100/soma):.2f}')
                print ('═══════════════════════════════════════════════ ')
    else:
        return
    print('\n')

## Menu para escolher a estatística visualizada

In [708]:
def menu_estatistica(dados:list):
    """ Função dedicada a exibir o menu das estatísticas propostas, sendo elas:
        Vendas por Gênero
        Vendas por Grupo
        Vendas por Categoria
        Vendas por Método de Pagamento
        Itens mais vendidos
    A função ficará em loop até o usuário solicitar a saída.
    Para cada uma das estatísticas, será chamada sua função correspondente
        para a realização dos cálculos e exibição na tela

    Args:
        dados (list): lista de dicionários com os dados de vendas
    """
    continuar_estatistica = True
    while continuar_estatistica:
        pergunta = """
        ╔═══════════════════════════════════════════╗
        ║  Qual Estatística você quer visualizar:   ║
        ╚═══════════════════════════════════════════╝"""
        print("""
        ╔═══════════════════════════════════════════╗
        ║          Lista de Estatíticas:            ║
        ╚═══════════════════════════════════════════╝""")
        print("        ║   1    Vendas por Gênero                  ║")
        print("        ║   2    Vendas por Grupo                   ║")
        print("        ║   3    Vendas por Categoria               ║")
        print("        ║   4    Vendas por Método de Pagamento     ║")
        print('        ║   5    Itens mais vendidos                ║')
        
        resp = validacao_respostas(pergunta, ['1','2','3','4','5'])
        if resp == '1':
            print("\nVendas por Gênero:\n")
            print ('═══════════════════════════════════════════════ ')
            estatistica_genero(dados)
        elif resp == '2':
            print("\n║            Vendas por Grupo:                ║\n║  Grupo A são clientes acima do Ticket Médio ║\n║  Grupo B são clientes abaixo do Ticket Medio║")
            estatistica_grupo_cliente(dados)
        elif resp == '3':
            print("\nVendas por Categoria:\n")
            print ('═══════════════════════════════════════════════ ')
            saida_estatistica(dados, 'Categoria')
        elif resp == '4':
            print("\nVendas por Método de Pagamento:\n")
            print ('═══════════════════════════════════════════════ ')
            saida_estatistica(dados, 'Metodo_Pagamento')
        elif resp == '5':
            pergunta = 'Quantos itens deseja visualizar?'
            qtd = validacao_numero(pergunta, int)
            print(f"\nOs {qtd} itens mais vendidos:\n")
            print ('═══════════════════════════════════════════════ ')
            saida_estatistica(dados, 'Item',qtd)
        pergunta = '''
        ╔═══════════════════════════════════════════╗
        ║    Deseja visualizar outra estatística    ║
        ╠═══════════════════════════════════════════╣
        ║             [S] Sim / [N] Não             ║
        ╚═══════════════════════════════════════════╝'''
        resp = validacao_respostas(pergunta, ['S','N'])
        if resp == 'N':
            continuar_estatistica = False  

# Função Principal

In [709]:
def main():
    """ Função principal:
    Primeiramente ocorre a seleção (ou não) dos dados de vendas.
    
    a função ficará em loop no menu até que o usuário desejar sair. As opções são:
    - Adicionar registro;
    - Remover registro;
    - editar registro;  
    - leitura de registros  
    - mostrar regra de negócios
    - realizar estatísticas (que também tem um menu próprio)
    - realizar filtros
    - exportar dados em formato json
    - trocar banco de dados
    - sair
    """
    
    # importando dados iniciais
    dados = menu_importar_dados()
    # iniciando o loop do menu
    
    continua = 'S'
    while continua =='S':
        print('\n')

        pergunta = """
===========================================
║       Qual ação deseja realizar?        ║
===========================================
| Adicionar um registro             --- A |
| Remover um registro               --- R |
| Editar um registro                --- E |
| Leitura de registros              --- L |
| Mostrar a regra de negócio        --- N |
| Realizar estatística              --- S |
| Realizar filtro                   --- F |
| Exportar dados para JSON          --- J |
| Trocar de banco de dados          --- T |
| Para sair do menu                 --- Q |
-------------------------------------------
        """
        opcao = validacao_respostas(pergunta, ['A','R','E','L','N','S','F','J','T','Q'])
        
        if   opcao == 'A': 
            dados = adicionar_dados_manualmente(dados)
        elif opcao == 'R': 
            dados = remover_arquivo(dados)
        elif opcao == 'E':
            dados = selecao_id_edicao(dados)
        elif opcao == 'L':
            menu_lista(dados)
        elif opcao == 'N':
            saida_ticket_medio(dados)
        elif opcao == 'S': 
            menu_estatistica(dados)
        elif opcao == 'F':
            opcoes_filtros(dados)
        elif opcao == 'J':
            exportar_json(dados)
        elif opcao == 'T':
            dados = menu_importar_dados()
        else:
            continua = 'N'
    print('Sistema Encerrado!')   
main()
    


    ╔═════════════════════════════════════╗
    ║    Bem Vindo à Gestão de Dados      ║
    ║ Que tipo de arquivo deseja importar:║
    ╠═════════════════════════════════════╣
    ║  1 	 CSV                      ║  
    ║  2 	 JSON                     ║
    ║  3 	 Não importar nada        ║ 
    ╚═════════════════════════════════════╝    
    
    
        | Lista de arquivos .JSON disponíveis |
	1	dados_vendas.json
	2	dados_vendas_traduzido.json
	3	teste_add_registro.json
	4	teste_remocao.json
	5	vendas_dinheiro.json

    ╔═════════════════════════════════════════════════════╗
    ║   Informe o número do arquivo que deseja importar   ║
    ╚═════════════════════════════════════════════════════╝
    



║       Qual ação deseja realizar?        ║
| Adicionar um registro             --- A |
| Remover um registro               --- R |
| Editar um registro                --- E |
| Leitura de registros              --- L |
| Mostrar a regra de negócio        --- N |
| Realizar estatística

IndexError: string index out of range